# Speech Feature Extraction using OpenSMILE (GeMapsv01b + ComParE config)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
from typing import List, Optional

import pandas as pd
from IPython.display import display
from tqdm.auto import tqdm

sys.path.append("..")
# opensmile
import opensmile

from sgs_utils.path_conf import loc_data_dir, speech_data_session_dir

# Extracting features

useful links:
* [opensmile config folder](https://github.com/audeering/opensmile/tree/v3.0.0/config)
* difference between GeMAPS versions [here](https://github.com/audeering/opensmile/blob/v3.0.0/config/gemaps/CHANGES.txt')

**note**: `eGeMAPS` is an _extended_ version of the GeMAPS

feature-level`
* `Functionals`: global segment based features (1 feature per segment)
* `LowLevelDescriptor`: sliding window features (1 feature per window)

In [ ]:
df_session = pd.read_parquet(loc_data_dir.joinpath("df_session_uuid.parquet"))

In [ ]:
# define the feature extraction configs
func_gemaps = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.Functionals,
)

lld_gemaps = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

# we will use the ComParE LLD to calculate frequency-based features on `F0final_sma`
lld_compare = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

In [ ]:
from multiprocessing import Pool
import traceback
from typing import Tuple

In [ ]:
def _extract_parse_smile_df(s: opensmile.Smile, f: Path) -> pd.DataFrame:
    df_feat = s.process_file(f)
    df_feat = df_feat.reset_index(drop=False)
    df_feat["file"] = df_feat["file"].astype("str")

    # df_feat["fileName"] = f.name
    df_feat["pic_name"] = f.name.split("__")[0]
    df_feat["time_str"] = f.name.split("__")[1].split('.')[0]
    df_feat["DB"] = f.parent.name
    df_feat["ID"] = f.parent.parent.name.split('__')[-1]
    return df_feat

def _extract_opensmile_f(file: Path) -> Tuple[pd.DataFrame, ...]:
    # calculate the global utterance features
    return (
        _extract_parse_smile_df(func_gemaps, f=file),
        _extract_parse_smile_df(lld_gemaps, f=file),
        _extract_parse_smile_df(lld_compare, f=file),
    )

out = None
with Pool(processes=3) as pool:
    wav_files = list(speech_data_session_dir.glob("*/*/*.wav"))
    results = pool.imap_unordered(_extract_opensmile_f, wav_files)
    results = tqdm(results, total=len(wav_files))
    try:
        out = [f for f in results]
    except:
        traceback.print_exc()
        pool.terminate()
    finally:
        pool.close()
        pool.join()

def _parse_concat_df(df_conc: pd.DataFrame) -> pd.DataFrame:
    df_conc["DB"] = df_conc["DB"].astype("category")
    df_conc["pic_name"] = df_conc["pic_name"].astype("category")
    df_conc["ID"] = df_conc["ID"].astype("category")
    return df_conc


df_gemaps_func = _parse_concat_df(pd.concat([o[0] for o in out], ignore_index=True))
df_gemaps_lld = _parse_concat_df(pd.concat([o[1] for o in out], ignore_index=True))
df_compare_lld = _parse_concat_df(pd.concat([o[2] for o in out], ignore_index=True))

del (
    out,
    _extract_opensmile_f,
    _parse_concat_df,
    _extract_parse_smile_df,
)

## Extract `F0`-range from the LLD's

In [ ]:
import numpy as np
from tsflex.features import FuncWrapper
from tsflex.features.utils import make_robust

In [ ]:
def quantile_nz(a: np.ndarray, q=List[float]) -> List[Optional[float]]:
    a_nz = a[a > 0]
    if len(a_nz):
        return np.quantile(a_nz, q=q)
    else:
        return [None] * len(q)


def nonzero_count(a: np.ndarray) -> int:
    return sum(a > 0)


def return_func_series_list(a: np.ndarray, f_list: List[FuncWrapper]) -> pd.Series:
    s = pd.Series(dtype="float64")
    for f in f_list:
        s = pd.concat([s, pd.Series(data=f(a), index=f.output_names)])
    return s.sort_index()


qs = sum([[1 - q, q] for q in [0, 0.01, 0.02, 0.03, 0.05, 0.1, 0.15, 0.2]], []) + [0.5]
display(str(qs))

### GeMAPS LLD

In [ ]:
df_gemaps_lld.filter(like="F0").columns
# logRelF0-H1-H2_sma3nz -> log freq difference between the harmonics
# SMA -> moving average window
# NZ -> no-zero

In [ ]:
# define the signal on which the function will be performed, and the functions
s_name = "F0semitoneFrom27.5Hz_sma3nz"

f_gemaps_lld_funcs: List[FuncWrapper] = [
    make_robust(
        FuncWrapper(quantile_nz, output_names=[s_name + f"_q={q}" for q in qs], q=qs),
        min_nb_samples=3,
        passthrough_nans=False,
    ),
    make_robust(FuncWrapper(nonzero_count, output_names=[f"{s_name}_nzcount"])),
]

# Apply the functions on each group
df_gemaps_lld_F0 = (
    df_gemaps_lld.groupby(by=["file"])[[s_name]]
    .apply(lambda x: return_func_series_list(x.values, f_list=f_gemaps_lld_funcs))
    .reset_index()
)
display(df_gemaps_lld_F0)

### ComPaRE LLD

In [ ]:
df_compare_lld.filter(like="F0").columns

In [ ]:
s_name = "F0final_sma"

f_compare_lld_funcs: List[FuncWrapper] = [
    make_robust(
        FuncWrapper(quantile_nz, output_names=[s_name + f"_q={q}" for q in qs], q=qs),
        min_nb_samples=3,
        passthrough_nans=False,
    ),
    make_robust(FuncWrapper(nonzero_count, output_names=[f"{s_name}_nzcount"])),
]

df_compare_lld_F0 = (
    df_compare_lld.groupby(by=["file"])[[s_name]]
    .apply(lambda x: return_func_series_list(x.values, f_list=f_compare_lld_funcs))
    .reset_index()
)
display(df_compare_lld_F0)

In [ ]:
del df_compare_lld, df_gemaps_lld

# Join into one big dataframe

In [ ]:
df_feat_tot = df_gemaps_func.merge(
    df_gemaps_lld_F0,
    on=["file"],
).merge(df_compare_lld_F0, on=["file"])

display(df_feat_tot)

df_feat_tot.to_parquet(loc_data_dir.joinpath("df_speech_feat_tot.parquet"), engine='fastparquet')

---

In [ ]:
df_feat_tot['file'].map(lambda x: '/'.join(x.split('/')[-3:]))

In [ ]:
df_feat_tot[['file', 'DB', 'pic_name', 'time_str', 'ID']]